<a href="https://colab.research.google.com/github/ansimhyang60/mini_project/blob/main/%EC%B9%B4%EB%93%9C%EC%97%B0%EC%B2%B4_%EC%98%88%EC%B8%A1_%EB%AA%A8%EB%8D%B8_%EC%A2%85%EB%A5%98_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 칼럼별 의미
- gender: 성별
- car: 차량 소유 여부
- reality: 부동산 소유 여부
- child_num: 자녀 수
- income_total: 연간 소득
- income_type: 소득 분류
							['Commercial associate', 'Working', 'State servant', 'Pensioner', 'Student']

- edu_type: 교육 수준
							['Higher education' ,'Secondary / secondary special', 'Incomplete higher', 'Lower secondary', 'Academic degree']

- family_type: 결혼 여부
							['Married', 'Civil marriage', 'Separated', 'Single / not married', 'Widow']

- house_type: 생활 방식
							['Municipal apartment', 'House / apartment', 'With parents',

							 'Co-op apartment', 'Rented apartment', 'Office apartment']

- DAYS_BIRTH: 출생일
							데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전에 태어났음을 의미

- DAYS_EMPLOYED: 업무 시작일
							데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 하루 전부터 일을 시작함을 의미

							양수 값은 고용되지 않은 상태를 의미함

- FLAG_MOBIL: 핸드폰 소유 여부
- work_phone: 업무용 전화 소유 여부
- phone: 전화 소유 여부
- email: 이메일 소유 여부
- occyp_type: 직업 유형
- family_size: 가족 규모
- begin_month: 신용카드 발급 월
							데이터 수집 당시 (0)부터 역으로 셈, 즉, -1은 데이터 수집일 한 달 전에 신용카드를 발급함을 의미

- credit: 사용자의 신용카드 대금 연체를 기준으로 한 신용도
							=> 낮을 수록 높은 신용의 신용카드 사용자를 의미함

In [ ]:
# !pip install catboost
# !pip install category_encoders
# !pip install pytorch_tabnet

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings, random
warnings.filterwarnings(action='ignore')

from sklearn.metrics import log_loss
from sklearn.preprocessing import StandardScaler
from category_encoders.ordinal import OrdinalEncoder
from sklearn.model_selection import StratifiedKFold

from sklearn.cluster import KMeans

from typing import Dict, Tuple, Union, List
import torch

from catboost import CatBoostClassifier, Pool
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from pytorch_tabnet.multitask import TabNetMultiTaskClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier


In [ ]:
def category_income(data: pd.DataFrame) -> pd.DataFrame:
  data["income_total"] = data["income_total"] / 10000
  conditions = [
    (data["income_total"].le(18)),
    (data["income_total"].gt(18) & data["income_total"].le(33)),
    (data["income_total"].gt(33) & data["income_total"].le(49)),
    (data["income_total"].gt(49) & data["income_total"].le(64)),
    (data["income_total"].gt(64) & data["income_total"].le(80)),
    (data["income_total"].gt(80) & data["income_total"].le(95)),
    (data["income_total"].gt(95) & data["income_total"].le(111)),
    (data["income_total"].gt(111) & data["income_total"].le(126)),
    (data["income_total"].gt(126) & data["income_total"].le(142)),
    (data["income_total"].gt(142)),
  ]
  choices = [i for i in range(10)]

  data["income_total"] = np.select(conditions, choices)
  return data

def load_dataset() -> Tuple[pd.DataFrame, pd.DataFrame]:
  train=pd.read_csv('/content/drive/MyDrive/CSV/train.csv')
  test=pd.read_csv('/content/drive/MyDrive/CSV/test.csv')
  #결측치 처리
  train.fillna('NaN', inplace=True)
  test.fillna('NaN', inplace=True)
  #이상치 처리
  train = train[(train['family_size'] <= 7)]
  train = train.reset_index(drop=True)
  #의미없는 변수 제거
  train.drop(['index', 'FLAG_MOBIL'], axis=1, inplace=True)
  test.drop(['index', 'FLAG_MOBIL'], axis=1, inplace=True)
  train['DAYS_EMPLOYED'] = train['DAYS_EMPLOYED'].map(lambda x: 0 if x > 0 else x)
  test['DAYS_EMPLOYED'] = test['DAYS_EMPLOYED'].map(lambda x: 0 if x > 0 else x)
  feats = ['DAYS_BIRTH', 'begin_month', 'DAYS_EMPLOYED']
  for feat in feats:
    train[feat]=np.abs(train[feat])
    test[feat]=np.abs(test[feat])

  #파생변수 생성
  for df in [train,test]:
    # before_EMPLOYED: 고용되기 전까지의 일수
    df['before_EMPLOYED'] = df['DAYS_BIRTH'] - df['DAYS_EMPLOYED']
    df['income_total_befofeEMP_ratio'] = df['income_total'] / df['before_EMPLOYED']
    df['before_EMPLOYED_m'] = np.floor(df['before_EMPLOYED'] / 30) - ((np.floor(df['before_EMPLOYED'] / 30) / 12).astype(int) * 12)
    df['before_EMPLOYED_w'] = np.floor(df['before_EMPLOYED'] / 7) - ((np.floor(df['before_EMPLOYED'] / 7) / 4).astype(int) * 4)

    #DAYS_BIRTH 파생변수- Age(나이), 태어난 월, 태어난 주(출생연도의 n주차)
    df['Age'] = df['DAYS_BIRTH'] // 365
    df['DAYS_BIRTH_m'] = np.floor(df['DAYS_BIRTH'] / 30) - ((np.floor(df['DAYS_BIRTH'] / 30) / 12).astype(int) * 12)
    df['DAYS_BIRTH_w'] = np.floor(df['DAYS_BIRTH'] / 7) - ((np.floor(df['DAYS_BIRTH'] / 7) / 4).astype(int) * 4)


    #DAYS_EMPLOYED_m 파생변수- EMPLOYED(근속연수), DAYS_EMPLOYED_m(고용된 달) ,DAYS_EMPLOYED_w(고용된 주(고용연도의 n주차))
    df['EMPLOYED'] = df['DAYS_EMPLOYED'] // 365
    df['DAYS_EMPLOYED_m'] = np.floor(df['DAYS_EMPLOYED'] / 30) - ((np.floor(df['DAYS_EMPLOYED'] / 30) / 12).astype(int) * 12)
    df['DAYS_EMPLOYED_w'] = np.floor(df['DAYS_EMPLOYED'] / 7) - ((np.floor(df['DAYS_EMPLOYED'] / 7) / 4).astype(int) * 4)

    #ability: 소득/(살아온 일수+ 근무일수)
    df['ability'] = df['income_total'] / (df['DAYS_BIRTH'] + df['DAYS_EMPLOYED'])

    #income_mean: 소득/ 가족 수
    df['income_mean'] = df['income_total'] / df['family_size']

    #ID 생성: 각 컬럼의 값들을 더해서 고유한 사람을 파악(*한 사람이 여러 개 카드를 만들 가능성을 고려해 begin_month는 제외함)
    df['ID'] = \
    df['child_num'].astype(str) + '_' + df['income_total'].astype(str) + '_' +\
    df['DAYS_BIRTH'].astype(str) + '_' + df['DAYS_EMPLOYED'].astype(str) + '_' +\
    df['work_phone'].astype(str) + '_' + df['phone'].astype(str) + '_' +\
    df['email'].astype(str) + '_' + df['family_size'].astype(str) + '_' +\
    df['gender'].astype(str) + '_' + df['car'].astype(str) + '_' +\
    df['reality'].astype(str) + '_' + df['income_type'].astype(str) + '_' +\
    df['edu_type'].astype(str) + '_' + df['family_type'].astype(str) + '_' +\
    df['house_type'].astype(str) + '_' + df['occyp_type'].astype(str)

  cols = ['child_num', 'DAYS_BIRTH', 'DAYS_EMPLOYED',]
  train.drop(cols, axis=1, inplace=True)
  test.drop(cols, axis=1, inplace=True)

  numerical_feats = train.dtypes[train.dtypes != "object"].index.tolist()
  numerical_feats.remove('credit')
  print("Number of Numerical features: ", len(numerical_feats))

  categorical_feats = train.dtypes[train.dtypes == "object"].index.tolist()
  print("Number of Categorical features: ", len(categorical_feats))

  for df in [train,test]:
    df['income_total'] = np.log1p(1+df['income_total'])

  encoder = OrdinalEncoder(categorical_feats)
  train[categorical_feats] = encoder.fit_transform(train[categorical_feats], train['credit'])
  test[categorical_feats] = encoder.transform(test[categorical_feats])

  train['ID'] = train['ID'].astype('int64')
  test['ID'] = test['ID'].astype('int64')

  #클러스터링 구성
  kmeans_train = train.drop(['credit'], axis=1)
  kmeans_train = kmeans_train[numerical_feats]
  kmeans = KMeans(n_clusters=36, random_state=42).fit(kmeans_train)
  train['cluster'] = kmeans.predict(kmeans_train)
  test['cluster'] = kmeans.predict(test[numerical_feats])

  numerical_feats.remove('income_total')
  scaler = StandardScaler()
  train[numerical_feats] = scaler.fit_transform(train[numerical_feats])
  test[numerical_feats] = scaler.transform(test[numerical_feats])

  #모델링 catboost
  n_est = 2000
  seed = 42
  n_fold = 15
  n_class = 3

  target = 'credit'
  X = train.drop(target, axis=1)
  y = train[target]
  X_test = test

In [ ]:
numerical_feats

['work_phone',
 'phone',
 'email',
 'family_size',
 'begin_month',
 'before_EMPLOYED',
 'income_total_befofeEMP_ratio',
 'before_EMPLOYED_m',
 'before_EMPLOYED_w',
 'before_EMPLOYED_w1',
 'Age',
 'DAYS_BIRTH_m',
 'DAYS_BIRTH_w',
 'DAYS_BIRTH_w1',
 'EMPLOYED',
 'DAYS_EMPLOYED_m',
 'DAYS_EMPLOYED_w',
 'DAYS_EMPLOYED_w1',
 'ability',
 'income_mean']

In [ ]:
categorical_feats

['gender',
 'car',
 'reality',
 'income_type',
 'edu_type',
 'family_type',
 'house_type',
 'occyp_type',
 'ID']

# cat_load_dataset()

In [ ]:
def cat_load_dataset() -> Tuple[pd.DataFrame, pd.DataFrame]:
  train=pd.read_csv('/content/drive/MyDrive/CSV/train.csv')
  test=pd.read_csv('/content/drive/MyDrive/CSV/test.csv')
  #결측치 처리
  train.fillna('NaN', inplace=True)
  test.fillna('NaN', inplace=True)
  #이상치 처리
  train = train[(train['family_size'] <= 7)]
  train = train.reset_index(drop=True)
  #의미없는 변수 제거
  train.drop(['index', 'FLAG_MOBIL'], axis=1, inplace=True)
  test.drop(['index', 'FLAG_MOBIL'], axis=1, inplace=True)
  train['DAYS_EMPLOYED'] = train['DAYS_EMPLOYED'].map(lambda x: 0 if x > 0 else x)
  test['DAYS_EMPLOYED'] = test['DAYS_EMPLOYED'].map(lambda x: 0 if x > 0 else x)
  feats = ['DAYS_BIRTH', 'begin_month', 'DAYS_EMPLOYED']
  for feat in feats:
    train[feat]=np.abs(train[feat])
    test[feat]=np.abs(test[feat])

  #income_ total
  train = category_income(train)
  test = category_income(test)

  #파생변수 생성
  for df in [train,test]:
    # before_EMPLOYED: 고용되기 전까지의 일수
    df['before_EMPLOYED'] = df['DAYS_BIRTH'] - df['DAYS_EMPLOYED']
    df['income_total_befofeEMP_ratio'] = df['income_total'] / df['before_EMPLOYED']
    df['before_EMPLOYED_m'] = np.floor(df['before_EMPLOYED'] / 30) - ((np.floor(df['before_EMPLOYED'] / 30) / 12).astype(int) * 12)
    df['before_EMPLOYED_w'] = np.floor(df['before_EMPLOYED'] / 7) - ((np.floor(df['before_EMPLOYED'] / 7) / 4).astype(int) * 4)

    #DAYS_BIRTH 파생변수- Age(나이), 태어난 월, 태어난 주(출생연도의 n주차)
    df['Age'] = df['DAYS_BIRTH'] // 365
    df['DAYS_BIRTH_m'] = np.floor(df['DAYS_BIRTH'] / 30) - ((np.floor(df['DAYS_BIRTH'] / 30) / 12).astype(int) * 12)
    df['DAYS_BIRTH_w'] = np.floor(df['DAYS_BIRTH'] / 7) - ((np.floor(df['DAYS_BIRTH'] / 7) / 4).astype(int) * 4)


    #DAYS_EMPLOYED_m 파생변수- EMPLOYED(근속연수), DAYS_EMPLOYED_m(고용된 달) ,DAYS_EMPLOYED_w(고용된 주(고용연도의 n주차))
    df['EMPLOYED'] = df['DAYS_EMPLOYED'] // 365
    df['DAYS_EMPLOYED_m'] = np.floor(df['DAYS_EMPLOYED'] / 30) - ((np.floor(df['DAYS_EMPLOYED'] / 30) / 12).astype(int) * 12)
    df['DAYS_EMPLOYED_w'] = np.floor(df['DAYS_EMPLOYED'] / 7) - ((np.floor(df['DAYS_EMPLOYED'] / 7) / 4).astype(int) * 4)

    #ability: 소득/(살아온 일수+ 근무일수)
    df['ability'] = df['income_total'] / (df['DAYS_BIRTH'] + df['DAYS_EMPLOYED'])

    #income_mean: 소득/ 가족 수
    df['income_mean'] = df['income_total'] / df['family_size']

    #ID 생성: 각 컬럼의 값들을 더해서 고유한 사람을 파악(*한 사람이 여러 개 카드를 만들 가능성을 고려해 begin_month는 제외함)
    df['ID'] = \
    df['child_num'].astype(str) + '_' + df['income_total'].astype(str) + '_' +\
    df['DAYS_BIRTH'].astype(str) + '_' + df['DAYS_EMPLOYED'].astype(str) + '_' +\
    df['work_phone'].astype(str) + '_' + df['phone'].astype(str) + '_' +\
    df['email'].astype(str) + '_' + df['family_size'].astype(str) + '_' +\
    df['gender'].astype(str) + '_' + df['car'].astype(str) + '_' +\
    df['reality'].astype(str) + '_' + df['income_type'].astype(str) + '_' +\
    df['edu_type'].astype(str) + '_' + df['family_type'].astype(str) + '_' +\
    df['house_type'].astype(str) + '_' + df['occyp_type'].astype(str)

  cols = ['child_num', 'DAYS_BIRTH', 'DAYS_EMPLOYED',]
  train.drop(cols, axis=1, inplace=True)
  test.drop(cols, axis=1, inplace=True)

  numerical_feats = train.dtypes[train.dtypes != "object"].index.tolist()
  numerical_feats.remove('credit')
  print("Number of Numerical features: ", len(numerical_feats))

  categorical_feats = train.dtypes[train.dtypes == "object"].index.tolist()
  print("Number of Categorical features: ", len(categorical_feats))

  for df in [train,test]:
    df['income_total'] = np.log1p(1+df['income_total'])

  encoder = OrdinalEncoder(categorical_feats)
  train[categorical_feats] = encoder.fit_transform(train[categorical_feats], train['credit'])
  test[categorical_feats] = encoder.transform(test[categorical_feats])

  train['ID'] = train['ID'].astype('int64')
  test['ID'] = test['ID'].astype('int64')

  #클러스터링 구성
  kmeans_train = train.drop(['credit'], axis=1)
  kmeans_train = kmeans_train[numerical_feats]
  kmeans = KMeans(n_clusters=36, random_state=42).fit(kmeans_train)
  train['cluster'] = kmeans.predict(kmeans_train)
  test['cluster'] = kmeans.predict(test[numerical_feats])

  numerical_feats.remove('income_total')
  scaler = StandardScaler()
  train[numerical_feats] = scaler.fit_transform(train[numerical_feats])
  test[numerical_feats] = scaler.transform(test[numerical_feats])

  #모델링 catboost
  n_est = 2000
  seed = 42
  n_fold = 15
  n_class = 3

  target = 'credit'
  X = train.drop(target, axis=1)
  y = train[target]
  X_test = test

# StratifiedKFold 기본모델, 성능은 0.665208

In [ ]:
skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
folds=[]
for train_idx, valid_idx in skfold.split(X, y):
        folds.append((train_idx, valid_idx))

cat_pred = np.zeros((X.shape[0], n_class))
cat_pred_test = np.zeros((X_test.shape[0], n_class))
cat_cols = ['income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type', 'ID']
for fold in range(n_fold):
  print(f'\n----------------- Fold {fold} -----------------\n')
  train_idx, valid_idx = folds[fold]
  X_train, X_valid, y_train, y_valid = X.iloc[train_idx], X.iloc[valid_idx], y[train_idx], y[valid_idx]
  train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
  valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

  model_cat = CatBoostClassifier(n_estimators=n_est, random_seed=seed)  #n_estimators=n_est, random_seed=seed  랜덤시드 고정
  model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=210, verbose=100)

  cat_pred[valid_idx] = model_cat.predict_proba(X_valid)
  cat_pred_test += model_cat.predict_proba(X_test) / n_fold
  print(f'CV Log Loss Score: {log_loss(y_valid, cat_pred[valid_idx]):.6f}')

print(f'\tLog Loss: {log_loss(y, cat_pred):.6f}')


TypeError: ignored

# stratified_kfold_cat 모델

In [ ]:
# Catboost
def stratified_kfold_cat(
    params: Dict[str, Union[int, float, str, List[str]]],
    n_fold: int,
    X: pd.DataFrame,
    y: pd.DataFrame,
    X_test: pd.DataFrame,
) -> Tuple[np.ndarray, np.ndarray]:
    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    splits = folds.split(X, y)
    cat_oof = np.zeros((X.shape[0], 3))
    cat_preds = np.zeros((X_test.shape[0], 3))
    cat_cols = [c for c in X.columns if X[c].dtypes == "int64"]

    for fold, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {fold} ============\n")
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
        valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

        model = CatBoostClassifier(**params, n_estimators=2000, random_seed=42 )

        model.fit(
            train_data,
            eval_set=valid_data,
            early_stopping_rounds=210,
            use_best_model=True,
            verbose=100,
        )

        cat_oof[valid_idx] = model.predict_proba(X_valid)
        cat_preds += model.predict_proba(X_test) / n_fold

    log_score = log_loss(y, cat_oof)
    print(f"Log Loss Score: {log_score:.5f}\n")
    return cat_oof, cat_preds

# cat Train
- cat 하이퍼파라미터튜닝(optuna 라이브러리)

In [ ]:
# train_cat = cat_load_dataset()
# test_cat = cat_load_dataset()
# X = train_cat
# y = train_cat["credit"]
# X_test = test_cat.copy()



# catboost 성능: log loss score: 0.66861
- 성능 낮음

In [ ]:
cat_params = {
    "learning_rate": 0.026612467217016746,
    "l2_leaf_reg": 0.3753065117824262,
    "max_depth": 8,
    "bagging_temperature": 1,
    "min_data_in_leaf": 57,
    "max_bin": 494,
    "random_state": 42,
    "eval_metric": "MultiClass",
    "loss_function": "MultiClass",
    "od_type": "Iter",
    "od_wait": 500,
    "iterations": 10000,
    "cat_features": [
       'gender',
       'car',
       'reality',
       'income_type',
       'edu_type',
       'family_type',
       'house_type',
       'occyp_type',
       'ID',],
}

cat_oof, cat_preds = stratified_kfold_cat(cat_params, 10, X, y, X_test)

# Light GBM

In [ ]:
# Light GBM
def stratified_kfold_lgbm(
    params: Dict[str, Union[int, float, str]],
    n_fold: int,
    X: pd.DataFrame,
    y: pd.DataFrame,
    X_test: pd.DataFrame,
) -> Tuple[np.ndarray, np.ndarray]:
    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    splits = folds.split(X, y)
    lgb_oof = np.zeros((X.shape[0], 3))
    lgb_preds = np.zeros((X_test.shape[0], 3))

    for fold, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {fold} ============\n")
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        pre_model = LGBMClassifier(**params)

        pre_model.fit(
            X_train,
            y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)],
            early_stopping_rounds=100,
            verbose=100,
        )
        params2 = params.copy()
        params2["learning_rate"] = params["learning_rate"] * 0.1

        model = LGBMClassifier(**params2)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)],
            early_stopping_rounds=100,
            verbose=100,
            init_model=pre_model,
        )
        lgb_oof[valid_idx] = model.predict_proba(X_valid)
        lgb_preds += model.predict_proba(X_test) / n_fold

    log_score = log_loss(y, lgb_oof)
    print(f"Log Loss Score: {log_score:.5f}")

    return lgb_oof, lgb_preds


# Light GBM Train(optuna 사용) 모델평과: 0.66409
- 성능 가장 높음

In [ ]:
train, test = load_dataset()
X = train.drop("credit", axis=1)
y = train["credit"]
X_test = test.copy()

In [ ]:
lgb_params = {
    "eta": 0.023839252347297356,
    "reg_alpha": 5.998770177220496e-05,
    "reg_lambda": 0.07127674208132959,
    "max_depth": 18,
    "num_leaves": 125,
    "colsample_bytree": 0.4241631237880101,
    "subsample": 0.8876057928391585,
    "subsample_freq": 5,
    "min_child_samples": 5,
    "max_bin": 449,
    "random_state": 42,
    "boosting_type": "gbdt",
    "learning_rate": 0.05,
    "n_estimators": 10000,
    "objective": "multiclass",
    "metric": "multi_logloss",
}
lgbm_oof, lgbm_preds = stratified_kfold_lgbm(lgb_params, 10, X, y, X_test)

# XGB

In [ ]:
def stratified_kfold_xgb(
    params: Dict[str, Union[int, float, str]],
    n_fold: int,
    X: pd.DataFrame,
    y: pd.DataFrame,
    X_test: pd.DataFrame,
) -> Tuple[np.ndarray, np.ndarray]:

    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    splits = folds.split(X, y)
    xgb_oof = np.zeros((X.shape[0], 3))
    xgb_preds = np.zeros((X_test.shape[0], 3))

    for fold, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {fold} ============\n")
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]

        model = XGBClassifier(**params)
        model.fit(
            X_train,
            y_train,
            eval_set=[(X_train, y_train), (X_valid, y_valid)],
            early_stopping_rounds=100,
            verbose=100,
        )

        xgb_oof[valid_idx] = model.predict_proba(X_valid)
        xgb_preds += model.predict_proba(X_test) / n_fold

    log_score = log_loss(y, xgb_oof)
    print(f"Log Loss Score: {log_score:.5f}")

    return xgb_oof, xgb_preds


# XGB Train

In [ ]:
xgb_params = {
    "eta": 0.023839252347297356,
    "reg_alpha": 6.99554614267605e-06,
    "reg_lambda": 0.010419988953061583,
    "max_depth": 15,
    "max_leaves": 159,
    "colsample_bytree": 0.4515469593932409,
    "subsample": 0.7732694309118915,
    "min_child_weight": 5,
    "gamma": 0.6847131315687576,
    "random_state": 42,
    "n_estimators": 10000,
    "objective": "multi:softmax",
    "eval_metric": "mlogloss",
}
xgb_oof, xgb_preds = stratified_kfold_xgb(xgb_params, 10, X, y, X_test)

#랜덤포레스트

In [ ]:
#랜덤포레스트
def stratified_kfold_rf(
    params: Dict[str, Union[int, float, str, bool]],
    n_fold: int,
    X: pd.DataFrame,
    y: pd.DataFrame,
    X_test: pd.DataFrame,
) -> Tuple[np.ndarray, np.ndarray]:

    folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
    splits = folds.split(X, y)
    rf_oof = np.zeros((X.shape[0], 3))
    rf_preds = np.zeros((X_test.shape[0], 3))

    for fold, (train_idx, valid_idx) in enumerate(splits):
        print(f"============ Fold {fold} ============\n")
        X_train, X_valid = X.iloc[train_idx], X.iloc[valid_idx]
        y_train, y_valid = y.iloc[train_idx], y.iloc[valid_idx]
        model = RandomForestClassifier(**params)
        model.fit(
            X_train,
            y_train,
        )

        rf_oof[valid_idx] = model.predict_proba(X_valid)
        rf_preds += model.predict_proba(X_test) / n_fold
        print(f"Log Loss Score: {log_loss(y_valid, rf_oof[valid_idx]):.5f}")

    log_score = log_loss(y, rf_oof)
    print(f"Log Loss Score: {log_score:.5f}")

    return rf_oof, rf_preds

In [ ]:
rf_params = {
        "criterion": "gini",
        "n_estimators": 300,
        "min_samples_split": 10,
        "min_samples_leaf": 2,
        "max_features": "auto",
        "oob_score": True,
        "random_state": 42,
        "n_jobs": -1,
    }
rf_oof, rf_preds = stratified_kfold_rf(rf_params, 10, X, y, X_test)

In [ ]:
train, test = load_dataset()
train_x = train.drop("credit", axis = 1)
train_y = train['credit'].values

In [ ]:
train_pred = np.concatenate([cat_oof, lgbm_oof, xgb_oof, rf_oof], axis=1)
train_pred.shape

In [ ]:
test_pred = np.concatenate([cat_preds, lgbm_preds, xgb_preds, rf_preds], axis=1)
test_pred.shape

# Pytorch Tabular
- Stacking Ensembel을 사용하여 학습진행

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
n_fold = 10
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)
splits = folds.split(train_pred, train_y)
net_oof = np.zeros((train_pred.shape[0], 3))
net_preds = np.zeros((test_pred.shape[0], 3))
for fold, (train_idx, valid_idx) in enumerate(splits):
    print(f"============ Fold {fold} ============\n")
    X_train, X_valid = train_pred[train_idx], train_pred[valid_idx]
    y_train, y_valid = train_y[train_idx], train_y[valid_idx]
    model = TabNetMultiTaskClassifier(
            n_d=64, n_a=64, n_steps=1,
            lambda_sparse=1e-4,
            optimizer_fn=torch.optim.Adam,
            optimizer_params=dict(lr=2e-2),
            scheduler_params = {"gamma": 0.9, "step_size": 50},
            scheduler_fn=torch.optim.lr_scheduler.StepLR,
            mask_type="entmax",
            device_name=device
    )

    model.fit(
        X_train, y_train.reshape(-1,1),
        eval_set=[(X_valid, y_valid.reshape(-1,1))],
        max_epochs=100,
        batch_size=1024,
        eval_metric=["logloss"],
        virtual_batch_size=128,
        num_workers=1,
        drop_last=False
    )
    net_oof[valid_idx] = model.predict_proba(X_valid)
    net_preds += model.predict_proba(test_pred)[0] / n_fold
log_score = log_loss(train_y, net_oof)
print(f"Log Loss Score: {log_score:.5f}")

In [ ]:
submission = pd.read_csv(path+"sample_submission.csv")
submission.iloc[:, 1:] = net_preds
submission.to_csv("meta_ensemble_submit.csv", index=False)

# 생략하기

In [ ]:
# def plot_feature_importance(importance,names,model_type):

#     feature_importance = np.array(importance)
#     feature_names = np.array(names)

#     data={'feature_names':feature_names,'feature_importance':feature_importance}
#     fi_df = pd.DataFrame(data)

#     fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

#     plt.figure(figsize=(10,8))

#     sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])

#     plt.title(model_type + ' Feature Importance')
#     plt.xlabel('Feature Importance')
#     plt.ylabel('Feature Names')

In [ ]:
# plot_feature_importance(model_cat.get_feature_importance(),X_test.columns,'CATBOOST')

In [ ]:
# fig, ax=plt.subplots() #히트맵, annot=true 컬럼명 출력
# fig.set_size_inches(20,10)
# sns.heatmap(train.corr(), annot=True, cmap='RdBu_r')  #각각의 공간에 숫자를 넣는 것을 annot=Ture

In [ ]:
# fig, ax=plt.subplots() #히트맵, annot=true 컬럼명 출력
# fig.set_size_inches(20,10)
# sns.heatmap(test.corr(), annot=True, cmap='RdBu_r')  #각각의 공간에 숫자를 넣는 것을 annot=Ture

In [ ]:
# import matplotlib as mpl
# import matplotlib.pyplot as plt
# import seaborn as sns
# from scipy import stats

# %matplotlib inline
# #그래프가 출려되지 않은 오류를 대처하기 위한 코드

# plt.style.use('ggplot')#그래프에서 범위를 격자로 보기

In [ ]:
# plt.subplots(figsize = (8,8))
# plt.pie(train['credit'].value_counts(),labels = train['credit'].value_counts().index,
# autopct="%.2f%%", shadow = True, startangle=90) # 비율 시작이 12시부터 시작.
# plt.show

# train, test 합친 데이터

In [ ]:
# import pandas as pd
# import numpy as np
# data_pension=pd.read_csv('/content/drive/MyDrive/Colab Notebooks/머신러닝프로젝트2조/CSV/data-Pension.csv')
# data_pension

In [ ]:
# data_pension.isnull().sum()

In [ ]:
# data_pension.fillna('NaN', inplace=True)

In [ ]:
# data_pension = data_pension[(data_pension['family_size'] <= 7)]
# data_pension = data_pension.reset_index(drop=True)

In [ ]:
# data_pension.columns

In [ ]:
# data_pension.drop(['FLAG_MOBIL', 'index', 'index-old'], axis=1)

In [ ]:
# data_pension['DAYS_EMPLOYED'] = data_pension['DAYS_EMPLOYED'].map(lambda x: 0 if x > 0 else x)

In [ ]:
# feats = ['DAYS_BIRTH', 'begin_month', 'DAYS_EMPLOYED']
# for feat in feats:
#     data_pension[feat]=np.abs(data_pension[feat])

In [ ]:
# for df in [data_pension]:
#     # before_EMPLOYED: 고용되기 전까지의 일수
#     df['before_EMPLOYED'] = df['DAYS_BIRTH'] - df['DAYS_EMPLOYED']
#     df['income_total_befofeEMP_ratio'] = df['income_total'] / df['before_EMPLOYED']
#     df['before_EMPLOYED_m'] = np.floor(df['before_EMPLOYED'] / 30) - ((np.floor(df['before_EMPLOYED'] / 30) / 12).astype(int) * 12)
#     df['before_EMPLOYED_w'] = np.floor(df['before_EMPLOYED'] / 7) - ((np.floor(df['before_EMPLOYED'] / 7) / 4).astype(int) * 4)

#     #DAYS_BIRTH 파생변수- Age(나이), 태어난 월, 태어난 주(출생연도의 n주차)
#     df['Age'] = df['DAYS_BIRTH'] // 365
#     df['DAYS_BIRTH_m'] = np.floor(df['DAYS_BIRTH'] / 30) - ((np.floor(df['DAYS_BIRTH'] / 30) / 12).astype(int) * 12)
#     df['DAYS_BIRTH_w'] = np.floor(df['DAYS_BIRTH'] / 7) - ((np.floor(df['DAYS_BIRTH'] / 7) / 4).astype(int) * 4)


#     #DAYS_EMPLOYED_m 파생변수- EMPLOYED(근속연수), DAYS_EMPLOYED_m(고용된 달) ,DAYS_EMPLOYED_w(고용된 주(고용연도의 n주차))
#     df['EMPLOYED'] = df['DAYS_EMPLOYED'] // 365
#     df['DAYS_EMPLOYED_m'] = np.floor(df['DAYS_EMPLOYED'] / 30) - ((np.floor(df['DAYS_EMPLOYED'] / 30) / 12).astype(int) * 12)
#     df['DAYS_EMPLOYED_w'] = np.floor(df['DAYS_EMPLOYED'] / 7) - ((np.floor(df['DAYS_EMPLOYED'] / 7) / 4).astype(int) * 4)

#     #ability: 소득/(살아온 일수+ 근무일수)
#     df['ability'] = df['income_total'] / (df['DAYS_BIRTH'] + df['DAYS_EMPLOYED'])

#     #income_mean: 소득/ 가족 수
#     df['income_mean'] = df['income_total'] / df['family_size']

#     #ID 생성: 각 컬럼의 값들을 더해서 고유한 사람을 파악(*한 사람이 여러 개 카드를 만들 가능성을 고려해 begin_month는 제외함)
#     df['ID'] = \
#     df['child_num'].astype(str) + '_' + df['income_total'].astype(str) + '_' +\
#     df['DAYS_BIRTH'].astype(str) + '_' + df['DAYS_EMPLOYED'].astype(str) + '_' +\
#     df['work_phone'].astype(str) + '_' + df['phone'].astype(str) + '_' +\
#     df['email'].astype(str) + '_' + df['family_size'].astype(str) + '_' +\
#     df['gender'].astype(str) + '_' + df['car'].astype(str) + '_' +\
#     df['reality'].astype(str) + '_' + df['income_type'].astype(str) + '_' +\
#     df['edu_type'].astype(str) + '_' + df['family_type'].astype(str) + '_' +\
#     df['house_type'].astype(str) + '_' + df['occyp_type'].astype(str)


In [ ]:
# cols = ['child_num', 'DAYS_BIRTH', 'DAYS_EMPLOYED',]
# data_pension.drop(cols, axis=1, inplace=True)

In [ ]:
# numerical_feats = data_pension.dtypes[data_pension.dtypes != "object"].index.tolist()
# print("Number of Numerical features: ", len(numerical_feats))

# categorical_feats = data_pension.dtypes[data_pension.dtypes == "object"].index.tolist()
# # print("Number of Categorical features: ", len(categorical_feats))

In [ ]:
# numerical_feats

In [ ]:
# categorical_feats

In [ ]:
# for df in []:
#     df['income_total'] = np.log1p(1+df['income_total'])

In [ ]:
# from category_encoders.ordinal import OrdinalEncoder

In [ ]:
# data_pension['ID']

In [ ]:
# encoder = OrdinalEncoder(categorical_feats)
# data_pension[categorical_feats] = encoder.fit_transform(data_pension[categorical_feats], data_pension['credit'])

In [ ]:
# data_pension['ID'] = data_pension['ID'].astype('int64')

In [ ]:
# kmeans_data_pension = data_pension.drop(['credit'], axis=1)
# kmeans_data_pension=kmeans_data_pension[numerical_feats]
# kmeans = KMeans(n_clusters=36, random_state=42).fit(kmeans_data_pension)
# data_pension['cluster'] = kmeans.predict(kmeans_data_pension)
# data_pension['cluster'] = kmeans.predict(data_pension[numerical_feats])

In [ ]:
# numerical_feats.remove('income_total')
# scaler = StandardScaler()
# data_pension[numerical_feats] = scaler.fit_transform(data_pension[numerical_feats])
# data_pension[numerical_feats] = scaler.transform(data_pension[numerical_feats])

In [ ]:
# n_est = 2000
# seed = 42
# n_fold = 15
# n_class = 3

# target = 'credit'
# X = data_pension.drop(target, axis=1)
# y = data_pension[target]
# X_test = test

In [ ]:
# skfold = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
# folds=[]
# for data_pension_idx, valid_idx in skfold.split(X, y):
#         folds.append((data_pension_idx, valid_idx))

# cat_pred = np.zeros((X.shape[0], n_class))
# cat_pred_test = np.zeros((X_test.shape[0], n_class))
# cat_cols = ['income_type', 'edu_type', 'family_type', 'house_type', 'occyp_type', 'ID']
# for fold in range(n_fold):
#   print(f'\n----------------- Fold {fold} -----------------\n')
#   data_pension_idx, valid_idx = folds[fold]
#   X_train, X_valid, y_train, y_valid = X.iloc[data_pension_idx], X.iloc[valid_idx], y[data_pension_idx], y[valid_idx]
#   train_data = Pool(data=X_train, label=y_train, cat_features=cat_cols)
#   valid_data = Pool(data=X_valid, label=y_valid, cat_features=cat_cols)

#   model_cat = CatBoostClassifier()
#   model_cat.fit(train_data, eval_set=valid_data, use_best_model=True, early_stopping_rounds=100, verbose=100)

# #   cat_pred[valid_idx] = model_cat.predict_proba(X_valid)
# #   cat_pred_test += model_cat.predict_proba(X_test) / n_fold
# #   print(f'CV Log Loss Score: {log_loss(y_valid, cat_pred[valid_idx]):.6f}')

# # print(f'\tLog Loss: {log_loss(y, cat_pred):.6f}')

In [ ]:
# def plot_feature_importance(importance,names,model_type):

#     feature_importance = np.array(importance)
#     feature_names = np.array(names)

#     data={'feature_names':feature_names,'feature_importance':feature_importance}
#     fi_df = pd.DataFrame(data)

#     fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

#     plt.figure(figsize=(10,8))

#     sns.barplot(x=fi_df['feature_importance'], y=fi_df['feature_names'])

#     plt.title(model_type + ' Feature Importance')
#     plt.xlabel('Feature Importance')
#     plt.ylabel('Feature Names')

In [ ]:
# plot_feature_importance(model_cat.get_feature_importance(),X_test.columns,'CATBOOST')

In [ ]:
# from sklearn.model_selection import train_test_split #모델 만들 때 비율: 0.2(독립변수 80%, 20%), 종속변수(80%, 20%)
# X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.2, random_state=156)

In [ ]:
# # RandomForestRegressor: RandomForest 알고리즘을 이용해서 값을 예측(회귀)
# from sklearn.ensemble import RandomForestRegressor
# rf_model=RandomForestRegressor(
#     random_state=0 #같은 값을 설정하면 다음에 실행할 때 같으 값을 선택
# )

In [ ]:
# rf_model=RandomForestRegressor(
#     random_state=0
# )

In [ ]:
# rf_model

In [ ]:
# rf_model.fit(X_train, y_train)

In [ ]:
# pred=rf_model.predict(X_test)

In [ ]:
# pred

In [ ]:
# from sklearn.metrics import mean_squared_error

In [ ]:
# print("MSE:", mean_squared_error(y_test, pred))

In [ ]:
# from xgboost import XGBRegressor

In [ ]:
# model_xgb=XGBRegressor(
#     n_estimators=150,
#     max_depth=5,
#     random_state=0,
#     min_child_weight=7,
#     learning_rate=0.1
# )

In [ ]:
# model_xgb.fit(X_train, y_train)

In [ ]:
# pred=model_xgb.predict(X_test)
# pred

In [ ]:
# print('MSE', mean_squared_error(pred, y_test))